# LangCahin Expression Language

## Basic example: prompt + model + output parser [¶](https://python.langchain.com/docs/expression_language/get_started#basic-example-prompt-model-output-parser)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = CommaSeparatedListOutputParser()

In [ ]:
chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "ice cream"})

### 1. Prompt

In [ ]:
from langchain.prompts.chat import ChatPromptValue

In [ ]:
input_dict: dict = {"topic": "Ice cream"}
prompt_value: ChatPromptValue = prompt.invoke(input_dict)
print(prompt_value)

In [ ]:
# Return prompt as a list of messages.
prompt_value.to_messages()

In [ ]:
# Return prompt as string.
prompt_value.to_string()

### 2. Model

In [ ]:
from langchain.schema.messages import AIMessage

In [ ]:
message: AIMessage = model.invoke(prompt_value)
message

In [ ]:
from langchain.llms import OpenAI
from langchain.llms import openai

In [ ]:
llm: openai.OpenAI = OpenAI(model="gpt-3.5-turbo-instruct")
type(llm)

### 3. Output parser

In [ ]:
output_parser.invoke(message)

## RAG 검색 예

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [ ]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")